<a href="https://colab.research.google.com/github/chetools/CVE2005/blob/main/FrictionFactor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from plotly.subplots import make_subplots
from scipy.optimize import root

In [2]:
rho = 1e3 #kg/m3
mu = 1e-3 #Pa s
g = 9.81 #m/s2
D = 0.1 #m, all pipes have same diameter in network
eD = 1e-3

In [3]:
def churchill_f(Re, eD):
    t2 = (37530./Re)**16
    t1 = (-2.457*np.log((7./Re)**0.9 + 0.27*eD) )**16
    return 8.*( (8./Re)**12 + (t1+t2)**(-1.5)  )**(1/12)

In [4]:
Re = np.logspace(np.log10(500),8,100)
eDs = np.logspace(-6, np.log10(.05),10)

In [5]:
fig = make_subplots()

for eD in eDs:
    fig.add_scatter(x=Re, y=churchill_f(Re, eD), mode='lines',name = f'eD={eD}')
fig.update_xaxes(type="log")
fig.update_yaxes(type="log")
fig.update_layout(width=800, height=600, template='plotly_dark')

In [6]:
#Q (flow) is positive for a positive headDrop
def headDrop(Q, L, D=D, eD=eD):
    sign = np.sign(Q)
    Q=np.abs(Q)
    V=4.*Q/np.pi/(D**2)
    Re = rho*V*D/mu
    if Re < 1e-10:
        return 0.

    res = churchill_f(Re, eD)*L*(V**2)/(g*D*2)
    return sign*res

In [7]:
headDrop(0.1, 100.)

591.1763691062081

In [8]:
#dh head drop (decrease in head)
def Q(dh, L, D=D, eD=eD):
    sign = np.sign(dh)
    dh = np.abs(dh)
    if dh<1e-10:
        return 0.

    def bernoulli(V):
        Re = rho*V*D/mu
        return dh - churchill_f(Re, eD)*L*(V**2)/(g*D*2)

    Vguess = (dh*2*g*D/0.05/L)**0.5
    V= root(bernoulli, Vguess).x
    return (sign*(np.pi*D**2)/4*V)[0]

In [9]:
Q(591.17, 100.)

0.09999946116398149

In [23]:
zs=np.array([10., 5., 0., 0., 0., 0.])
external_idx = np.array([1,6]) - 1 #Python indexing starts at 0
pipe_network = np.array([[1,2, 10.],
                         [2,3, 8.],
                         [2,4, 8.],
                         [2,5, 20.],
                         [3,5, 6.],
                         [4,5, 7.],
                         [4,6, 1.],
                         [5,6, 1.]])
pipe_lengths = pipe_network[:,2]
N_pipes = np.size(pipe_lengths)
start_idxs = pipe_network[:,0].astype(int)-1
end_idxs = pipe_network[:,1].astype(int)-1
N_nodes=np.max(end_idxs)+1

internal_idx = np.setdiff1d(np.arange(N_nodes), external_idx)

In [11]:
P=np.zeros(N_nodes)
P[external_idx]=np.array([3e5, 0.])
avg_external_head = np.mean(P[external_idx]/rho/g + zs[external_idx])
Pguess_internal=rho*g*(avg_external_head - zs[internal_idx])

In [39]:

def calcFlows(internal_P):
    netflows = np.zeros(N_nodes)
    P[internal_idx]=internal_P
    heads = P/rho/g + zs
    flows = np.zeros(N_pipes)
    for pipe_idx in range(N_pipes):
        dh = heads[start_idxs[pipe_idx]] - heads[end_idxs[pipe_idx]]
        flows[pipe_idx] = Q(dh, pipe_lengths[pipe_idx])
        np.add.at(netflows,start_idxs[pipe_idx],-flows[pipe_idx])
        np.add.at(netflows,end_idxs[pipe_idx], flows[pipe_idx])
    return netflows, flows

In [40]:
calcFlows(Pguess_internal)

(array([-0.05857373,  0.05857373,  0.        , -0.18528777, -0.18528777,
         0.37057554]),
 array([0.05857373, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.18528777, 0.18528777]))

In [41]:
Pinternal = root(lambda Pguess_internal: calcFlows(Pguess_internal)[0][internal_idx], Pguess_internal).x

In [42]:
calcFlows(Pinternal)

(array([-7.67309193e-02, -4.05925293e-16, -6.93889390e-18,  2.16313079e-13,
        -2.26312025e-13,  7.67309193e-02]),
 array([ 0.07673092,  0.02417894,  0.03232643,  0.02022555,  0.02417894,
        -0.00492364,  0.03725007,  0.03948085]))

In [13]:
pipe_network = np.array([[1,2, 10.],
                         [2,3, 8.],
                         [2,4, 8.],
                         [2,5, 20.],
                         [3,5, 6.],
                         [4,5, 7.],
                         [4,6, 1.],
                         [5,6, 1.]])

In [43]:
P[internal_idx]=Pinternal

In [44]:
P

array([300000.        ,   7541.87321482,  29423.54794629,   8054.42742887,
         9047.30399489,      0.        ])